This notebook uses the Velodrome architecture on the Druid dataset, for baseline comparison. The input data is the annotated mutation profile, from the Foundation One report.

Velodrome reference: https://www.nature.com/articles/s42256-021-00408-w

Before running this notebook, train Velodrome using the command from `src/baselines` directory:

`nohup python velodrome.py > velodrome_sample0.log &`

In [1]:
import pandas as pd
import numpy as np


import sys
sys.path.append("../src/baselines/")

# In[3]:


import datetime
import logging
import os
import time
import torch
import random
import pickle


# In[4]:


from torch import nn
from torch.nn import functional as F

from functools import cached_property

from torch.nn import Linear, ReLU, Sequential
from sklearn.metrics import average_precision_score, ndcg_score, roc_auc_score
from sklearn.model_selection import train_test_split


# In[5]:

from seaborn import scatterplot


# In[6]:


from FuncVelov3 import *


# In[7]:

sample_id = 2
# arguments class for hyperparameters like weight decay, learning rate etc
class arguments():
    def __init__(self):
        self.seed = 42
        self.ldr = 0.5 # dropout
        self.hd = 1 # for network architecture
        self.bs = 64 
        self.wd = 0.5 # weight decay
        self.wd1 = 0.1
        self.wd2 = 0.1
        self.lr = 0.001 # learning rate
        self.lr1 = 0.005
        self.lr2 = 0.005
        self.lam1 = 0.005
        self.lam2 = 0.005
        self.epoch = 30
        self.gpu = 0
        self.save_logs = f"../src/baselines/model_checkpoints/velodrome_logs_{sample_id}/"
        self.save_models = f"../src/baselines/model_checkpoints/velodrome_models_{sample_id}/"
        self.save_results = f"../src/baselines/model_checkpoints/velodrome_results_{sample_id}/"
        os.makedirs(self.save_logs, exist_ok=True)
        os.makedirs(self.save_models, exist_ok=True)
        os.makedirs(self.save_results, exist_ok=True)
args = arguments()


# In[8]:


torch.manual_seed(args.seed)
random.seed(args.seed)
np.random.seed(args.seed)

# To avoid randomness in DataLoaders - https://pytorch.org/docs/stable/notes/randomness.html
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
g = torch.Generator()
g.manual_seed(0)

# In[9]:


torch.multiprocessing.set_sharing_strategy('file_system')


In [2]:
# read w1 and w2 from saved pickle
weights_file = args.save_models + "weights.pickle"
with open(weights_file, 'rb') as f:
    weights_dict = pickle.load(f)

In [3]:
len(weights_dict)

7

In [4]:
# Velodrome definition from https://github.com/hosseinshn/Velodrome
def VelodromeNetwork(args, X):
    IE_dim = X.shape[1]

    class Net1(nn.Module):
        def __init__(self, args):
            super(Net1, self).__init__()

            self.features = nn.Sequential(
                nn.Linear(IE_dim, 512),
                nn.BatchNorm1d(512),
                nn.ReLU(),
                nn.Dropout(args.ldr),
                nn.Linear(512, 128),
                nn.Sigmoid()) 

        def forward(self, x):
            out = self.features(x)
            return out     
    
    class Net2(nn.Module):
        def __init__(self, args):
            super(Net2, self).__init__()

            self.features = torch.nn.Sequential(
                nn.Linear(IE_dim, 256),
                nn.BatchNorm1d(256),
                nn.ReLU(),
                nn.Dropout(args.ldr),
                nn.Linear(256, 256),
                nn.Sigmoid()
                ) 

        def forward(self, x):
            out = self.features(x)
            return out            

    class Net3(nn.Module):
        def __init__(self, args):
            super(Net3, self).__init__()    

            self.features = torch.nn.Sequential(
                nn.Linear(IE_dim, 128),
                nn.BatchNorm1d(128),
                nn.ReLU(),
                nn.Dropout(args.ldr),
                nn.Linear(128, 128),
                nn.BatchNorm1d(128),
                nn.ReLU(),
                nn.Dropout(args.ldr),
                nn.Linear(128, 128),
                nn.Sigmoid()
                )                        

        def forward(self, x):
            out = self.features(x)
            return out

    class Net4(nn.Module):
        def __init__(self, args):
            super(Net4, self).__init__()    

            self.features = torch.nn.Sequential(
                nn.Linear(IE_dim, 64),
                nn.BatchNorm1d(64),
                nn.ReLU(),
                nn.Dropout(args.ldr),
                nn.Linear(64, 64),
                nn.BatchNorm1d(64),
                nn.ReLU(),
                nn.Dropout(args.ldr),
                nn.Linear(64, 64),
                nn.Sigmoid()
                )                        

        def forward(self, x):
            out = self.features(x)
            return out        
        
    if args.hd == 1:
        Model = Net1(args)
    elif args.hd == 2:
        Model = Net2(args)
    elif args.hd == 3:
        Model = Net3(args)
    elif args.hd == 4:
        Model = Net4(args)        

    class Pred(nn.Module):
        def __init__(self, args):
            super(Pred, self).__init__()
            if args.hd == 1: 
                dim = 128            
            if args.hd == 2: 
                dim = 256
            if args.hd == 3:
                dim = 128
            if args.hd == 4:
                dim = 64                
            self.pred = torch.nn.Sequential(
                nn.Linear(dim, 1)) 

        def forward(self, x):
            out = self.pred(x)
            return out     
    torch.manual_seed(args.seed)    
    Predict_1 = Pred(args)
    torch.manual_seed(args.seed*2)
    Predict_2 = Pred(args)
    torch.manual_seed(args.seed)
        
    return Model, Predict_1, Predict_2

In [5]:
# Class definition for running test bed evaluation
# start with gene exp, one drug per model, 385 instances then average out for test bed eval
# Here we use one network for one drug to test
class VelodromeOneDrug(nn.Module):
    def __init__(self, drug_name, args, X1_train):
        super(VelodromeOneDrug, self).__init__()
        self.X1_train = X1_train
        self.drug_name = drug_name.replace("/", "_")
        self.model, self.pred1, self.pred2 = VelodromeNetwork(args=args, X=self.X1_train)
        self.w1 = weights_dict[drug_name]["w1"]
        self.w2 = weights_dict[drug_name]["w2"]
        
                                   
    def forward(self, X):
        self.model.load_state_dict(torch.load(os.path.join(args.save_models, f'{self.drug_name}Best_Model.pt')))
        self.pred1.load_state_dict(torch.load(os.path.join(args.save_models, f'{self.drug_name}Best_Pred1.pt')))
        self.pred2.load_state_dict(torch.load(os.path.join(args.save_models, f'{self.drug_name}Best_Pred2.pt')))
        self.model.eval()
        self.pred1.eval()
        self.pred2.eval()
        ws = [torch.mean(torch.stack(self.w1)), torch.mean(torch.stack(self.w2))]
        w_n = torch.nn.functional.softmax(torch.stack(ws), dim=None)
        w1 = w_n[0]
        w2 = w_n[1]
        
        TX_val = torch.tensor(
            X,
            dtype=torch.float,
        )

        fx_val = self.model(TX_val)
        pred_1 = self.pred1(fx_val)
        pred_2 = self.pred2(fx_val)
        pred_val = w1*pred_1+w2*pred_2
        return pred_val

In [6]:
tcga_test_df = pd.read_csv(f"../data/diffusion_pretraining/tcga_diffusion_test_sample{sample_id}.csv", index_col=0)
tcga_test_df.shape

(120, 7776)

In [7]:
patient_responses_df = pd.read_csv("/data/ajayago/copied_from_cdal1/ajayago_home_folder/processed/TCGA_drug_response_010222.csv")
patient_responses_df

,patient.arr,drug.name,response,response_cat,drug
0,TCGA-G2-A2EC,Methotrexate,Partial Response,1,METHOTREXATE
1,TCGA-G2-A2EC,Doxorubicin,Partial Response,1,DOXORUBICIN
2,TCGA-G2-A2EC,Vinblastine,Partial Response,1,VINBLASTINE
3,TCGA-G2-A2EC,Cisplatin,Partial Response,1,CISPLATIN
4,TCGA-G2-A2EJ,Paclitaxel,Stable Disease,0,PACLITAXEL
...,...,...,...,...,...
1244,TCGA-BG-A0VZ,Cisplatin,Complete Response,1,CISPLATIN
1245,TCGA-BG-A0VZ,Paclitaxel,Complete Response,1,PACLITAXEL
1246,TCGA-BG-A0VZ,Doxorubicin,Complete Response,1,DOXORUBICIN
1247,TCGA-BG-A0VT,Carboplatin,Complete Response,1,CARBOPLATIN


In [8]:
test_tcga_with_response = patient_responses_df[patient_responses_df["patient.arr"].isin(tcga_test_df.index)].reset_index(drop=True)
test_tcga_with_response

,patient.arr,drug.name,response,response_cat,drug
0,TCGA-G2-A2EF,Methotrexate,Partial Response,1,METHOTREXATE
1,TCGA-G2-A2EF,Vinblastine,Partial Response,1,VINBLASTINE
2,TCGA-G2-A2EF,Doxorubicin,Partial Response,1,DOXORUBICIN
3,TCGA-G2-A2EF,Cisplatin,Partial Response,1,CISPLATIN
4,TCGA-DK-A3IQ,Gemcitabine,Stable Disease,0,GEMCITABINE
...,...,...,...,...,...
271,TCGA-QS-A8F1,Carboplatin,Clinical Progressive Disease,0,CARBOPLATIN
272,TCGA-QS-A8F1,Paclitaxel,Clinical Progressive Disease,0,PACLITAXEL
273,TCGA-BG-A0VZ,Cisplatin,Complete Response,1,CISPLATIN
274,TCGA-BG-A0VZ,Paclitaxel,Complete Response,1,PACLITAXEL


In [9]:
predictions = []
y_true = []
drugs = []
for idx, row in test_tcga_with_response.iterrows():
    if row["drug"] in ["CISPLATIN","PACLITAXEL","5-FLUOROURACIL","DOCETAXEL","GEMCITABINE","DOXORUBICIN", "CYCLOPHOSPHAMIDE"]:
        m = VelodromeOneDrug(row["drug"], args, torch.randn((1, 7776))).to(torch.device("cuda:0"))
        inp = torch.from_numpy(tcga_test_df.loc[row["patient.arr"]].values.astype(np.float32)).view(1, -1).to(torch.device("cuda:0"))
        # print(inp.shape)
        predictions.append(nn.Sigmoid()(m(inp)).cpu().detach().numpy()[0][0])
        y_true.append(row["response_cat"])
        drugs.append(row["drug"])

/tmp/ipykernel_3626179/2784033225.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  w_n = torch.nn.functional.softmax(torch.stack(ws), dim=None)
/tmp/ipykernel_3626179/2784033225.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  TX_val = torch.tensor(


In [10]:
len(predictions), len(y_true)

(159, 159)

In [11]:
from sklearn.metrics import roc_auc_score, average_precision_score
auroc = roc_auc_score(y_true, predictions)
auprc = average_precision_score(y_true, predictions)

In [12]:
print(f"AUROC = {auroc}, AUPRC = {auprc}")

AUROC = 0.524308300395257, AUPRC = 0.7052939956697875


In [13]:
# per drug
res_df = pd.DataFrame(predictions, columns=["y_pred"])
res_df["y_true"] = y_true
res_df["drug"] = drugs

In [14]:
res_df

,y_pred,y_true,drug
0,0.714649,1,DOXORUBICIN
1,0.715069,1,CISPLATIN
2,0.718982,0,GEMCITABINE
3,0.717475,0,DOXORUBICIN
4,0.717695,0,CISPLATIN
...,...,...,...
154,0.718754,1,PACLITAXEL
155,0.718146,0,PACLITAXEL
156,0.717772,1,CISPLATIN
157,0.717277,1,PACLITAXEL


In [15]:
res_df.y_true.value_counts()

y_true
1    115
0     44
Name: count, dtype: int64

In [16]:
for d in res_df.drug.unique():
    try:
        subset_df = res_df[res_df.drug == d]
        auroc = roc_auc_score(subset_df["y_true"], subset_df["y_pred"])
        auprc = average_precision_score(subset_df["y_true"], subset_df["y_pred"])
        print(f"Drug {d} | AUROC = {auroc}, AUPRC = {auprc}")
        print(subset_df.shape)
        print(subset_df.y_true.value_counts())
    except:
        continue

Drug DOXORUBICIN | AUROC = 0.7272727272727272, AUPRC = 0.9218253968253969
(14, 3)
y_true
1    11
0     3
Name: count, dtype: int64
Drug CISPLATIN | AUROC = 0.5310344827586208, AUPRC = 0.7766118986522895
(39, 3)
y_true
1    29
0    10
Name: count, dtype: int64
Drug GEMCITABINE | AUROC = 0.4722222222222222, AUPRC = 0.5835016835016835
(12, 3)
y_true
0    6
1    6
Name: count, dtype: int64
Drug PACLITAXEL | AUROC = 0.5822222222222222, AUPRC = 0.7387470836983221
(34, 3)
y_true
1    25
0     9
Name: count, dtype: int64
Drug 5-FLUOROURACIL | AUROC = 0.588888888888889, AUPRC = 0.7697106130003478
(29, 3)
y_true
1    20
0     9
Name: count, dtype: int64
Drug CYCLOPHOSPHAMIDE | AUROC = 0.8125, AUPRC = 0.9757106608669109
(18, 3)
y_true
1    16
0     2
Name: count, dtype: int64
Drug DOCETAXEL | AUROC = 0.375, AUPRC = 0.5540990259740259
(13, 3)
y_true
1    8
0    5
Name: count, dtype: int64
